In [1]:
import pandas as pd
import numpy as np
import networkx as nx

df_cleaning = pd.read_excel('fin_cleansing.xlsx', 'Sheet1')
df = pd.read_excel('poc_fin_masked_lama.xlsx', 'Searah')
df_checkpoint = pd.read_excel('poc_fin_masked_lama.xlsx', 'Checkpoint')
df_uplink = pd.read_excel('uplink_ori_dummy.xlsx', 'Sheet3')
df_downlink = pd.read_excel('downlink_ori_dummy.xlsx', 'Sheet2')

In [2]:
#shortestpath uplink HSI
H = nx.DiGraph()

for i in range(len(df)):
  from1_element = df.iloc[i]['from1']
  to1_element = df.iloc[i]['to1']
  cost = df.iloc[i]['cost_link']
  if 'VPN' in from1_element or'VPN' in to1_element or 'TRANSIT' in from1_element or'TRANSIT' in to1_element:
    continue
#   elif 'MET9**KBL' in from1_element and 'MET9**RKT' in to1_element:
#     continue
#   elif 'MET9**RKT' in from1_element and 'MET9**KBL' in to1_element:
#     continue
  else:
    H.add_edge(from1_element,to1_element,weight=cost)

  #print("from: ", from1_element, "to: ", to1_element, "cost: ",cost)


def remove_edge(H, node_from, node_to):
    if H.has_edge(node_from, node_to):
        H.remove_edge(node_from, node_to)

for i in range(len(df_checkpoint)):
    for j in range(len(df_checkpoint)):
        remove_edge(H, df_checkpoint.iloc[i]['Checkpoint'], df_checkpoint.iloc[j-1]['Checkpoint'])


# shortest_path = nx.shortest_path(H, 'MET9**KBL', 'MET9**RKT', weight='weight')
# length = nx.shortest_path_length(H, 'OLT00**ABU*2DSU', 'MET9**KBL', weight='weight')
# print("Shortest Path:", shortest_path)
# print("total_cost:", length)

In [3]:
#shortestpath uplink VPN
V = nx.DiGraph()

for i in range(len(df)):
  from1_element = df.iloc[i]['from1']
  to1_element = df.iloc[i]['to1']
  cost = df.iloc[i]['cost_link']
  if 'BR' in from1_element or 'BR' in to1_element:
    continue
#   elif 'MET9**KBL' in from1_element and 'MET9**RKT' in to1_element:
#     continue
#   elif 'MET9**RKT' in from1_element and 'MET9**KBL' in to1_element:
#     continue
  else:
    V.add_edge(from1_element,to1_element,weight=cost)

  #print("from: ", from1_element, "to: ", to1_element, "cost: ",cost)

shortest_path = nx.shortest_path(V, 'MET9**RKT', 'CNFacebook', weight='weight')
length = nx.shortest_path_length(V, 'OLT00**ABU*2DSU', 'MET9**KBL', weight='weight')
print("Shortest Path:", shortest_path)
print("total_cost:", length)

Shortest Path: ['MET9**RKT', 'PE2**RKT*VPN', 'P**RKT', 'P**KBL', 'PE**KBL*VPN', 'MET4**KBL', 'PE**KBL*HSI', 'CNFacebook']
total_cost: 300


In [4]:
#shortestpath uplink Transit
T = nx.DiGraph()

for i in range(len(df)):
  from1_element = df.iloc[i]['from1']
  to1_element = df.iloc[i]['to1']
  cost = df.iloc[i]['cost_link']
  if 'BR' in from1_element or 'BR' in to1_element or 'VPN' in from1_element or'VPN' in to1_element:
    continue
#   elif 'MET9**KBL' in from1_element and 'MET9**RKT' in to1_element:
#     continue
#   elif 'MET9**RKT' in from1_element and 'MET9**KBL' in to1_element:
#     continue
  else:
    T.add_edge(from1_element,to1_element,weight=cost)

  #print("from: ", from1_element, "to: ", to1_element, "cost: ",cost)

shortest_path = nx.shortest_path(T, 'MET9**RKT', 'CGW*.PNK', weight='weight')
length = nx.shortest_path_length(T, 'OLT00**ABU*2DSU', 'MET9**KBL', weight='weight')
print("Shortest Path:", shortest_path)
print("total_cost:", length)

Shortest Path: ['MET9**RKT', 'PE2**RKT*TRANSIT', 'P**RKT', 'P*D6*ULN', 'P*D7*PNK', 'CGW*.PNK']
total_cost: 300


In [5]:
path_hsi = []
total_cost_hsi = []
checkpoint_hsi = []

path_vpn = []
total_cost_vpn = []

path_transit = []
total_cost_transit = []

def check_path_exists(H, node_from, node_to):
    return nx.has_path(H, node_from, node_to)

for i in range(len(df_uplink)):
    from1 = df_uplink.iloc[i]['olt1']
    to1 = df_uplink.iloc[i]['to1']
    produk = df_uplink.iloc[i]['produk']
    metro = 999999
    if produk == 'hsi':
        for chk in range(len(df_checkpoint)):
            if check_path_exists(H, from1, df_checkpoint.iloc[chk]['Checkpoint']):
                if nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight') < metro:
                    metro = nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight')
                    checkpoint = df_checkpoint.iloc[chk]['Checkpoint']
            else:
                continue       
        path_hsi.append(nx.shortest_path(H, from1, checkpoint, weight='weight'))
        total_cost_hsi.append(metro)
        checkpoint_hsi.append(checkpoint)

    elif produk == 'vpn':
        path_vpn.append(nx.shortest_path(V, from1,to1, weight='weight'))
        total_cost_vpn.append(nx.shortest_path_length(V, from1, to1, weight='weight'))
        
    elif produk == 'transit':
        path_transit.append(nx.shortest_path(T, from1,to1, weight='weight'))
        total_cost_transit.append(nx.shortest_path_length(T, from1, to1, weight='weight'))

df_baru_hsi = df_uplink[df_uplink['produk'] == 'hsi']
df_baru_hsi = df_baru_hsi.drop(columns=['Unnamed: 0'])
df_baru_hsi['path'] = path_hsi
df_baru_hsi['total_cost'] = total_cost_hsi
df_baru_hsi['checkpoint'] = checkpoint_hsi

df_baru_vpn = df_uplink[df_uplink['produk'] == 'vpn']
df_baru_vpn = df_baru_vpn.drop(columns=['Unnamed: 0'])
df_baru_vpn['path'] = path_vpn
df_baru_vpn['total_cost'] = total_cost_vpn

df_baru_transit = df_uplink[df_uplink['produk'] == 'transit']
df_baru_transit = df_baru_transit.drop(columns=['Unnamed: 0'])
df_baru_transit['path'] = path_transit
df_baru_transit['total_cost'] = total_cost_transit

In [6]:
path = []
costs = []
for i in range(len(df_baru_hsi)):
    sp = nx.shortest_path(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['to1'] , weight='weight')
    tc = nx.shortest_path_length(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['to1'] , weight='weight')
    sp.pop(0)
    
    path.append(df_baru_hsi.iloc[i]['path'] + sp)
    costs.append(df_baru_hsi.iloc[i]['total_cost'] + tc)
    
df_akhir_hsi = df_baru_hsi.copy() 
df_akhir_hsi = df_akhir_hsi.drop('path', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('total_cost', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('checkpoint', axis=1)
df_akhir_hsi['path'] = path
df_akhir_hsi['total_cost'] = costs

df_combined_uplink = pd.concat([df_akhir_hsi, df_baru_vpn, df_baru_transit])
df_combined_uplink

,olt1,to1,service,bw_up,produk,path,total_cost
0,OLT00**KPS*3,CNFacebook,MOBILE,0.023,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30000.0
1,OLT00**KPS*3,CNGoogle,MOBILE,0.023,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30000.0
2,OLT00**KPS*3,CGW*.MDO,MOBILE,0.045,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30819.0
3,OLT00**KPS*3,CGW*.BDS,MOBILE,0.045,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30010.0
4,OLT00**KPS*3,CNConversant,MOBILE,0.023,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30000.0
5,OLT00**KPS*3,CNNetflix,MOBILE,0.023,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30000.0
6,OLT00**KPS*3,CNNetflix,WHOLESALE,0.011,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30000.0
7,OLT00**KPS*3,CNConversant,WHOLESALE,0.011,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30000.0
8,OLT00**KPS*3,CGW*.BDS,WHOLESALE,0.034,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30010.0
9,OLT00**KPS*3,CGW*.MDO,WHOLESALE,0.034,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL, BR2**KBL, ...",30819.0


In [7]:
path_hsi = []
total_cost_hsi = []
checkpoint_hsi = []

path_vpn = []
total_cost_vpn = []

path_transit = []
total_cost_transit = []

def check_path_exists(H, node_from, node_to):
    return nx.has_path(H, node_from, node_to)

for i in range(len(df_downlink)):
    from1 = df_downlink.iloc[i]['olt1']
    to1 = df_downlink.iloc[i]['from1']
    produk = df_downlink.iloc[i]['produk']
    metro = 999999
    if produk == 'hsi':
        for chk in range(len(df_checkpoint)):
            if check_path_exists(H, from1, df_checkpoint.iloc[chk]['Checkpoint']):
                if nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight') < metro:
                    metro = nx.shortest_path_length(H, from1, df_checkpoint.iloc[chk]['Checkpoint'], weight='weight')
                    checkpoint = df_checkpoint.iloc[chk]['Checkpoint']
            else:
                continue       
        path_hsi.append(nx.shortest_path(H, from1, checkpoint, weight='weight'))
        total_cost_hsi.append(metro)
        checkpoint_hsi.append(checkpoint)

    elif produk == 'vpn':
        path_vpn.append(nx.shortest_path(V, from1,to1, weight='weight'))
        total_cost_vpn.append(nx.shortest_path_length(V, from1, to1, weight='weight'))
        
    elif produk == 'transit':
        path_transit.append(nx.shortest_path(T, from1,to1, weight='weight'))
        total_cost_transit.append(nx.shortest_path_length(T, from1, to1, weight='weight'))

df_baru_hsi = df_downlink[df_downlink['produk'] == 'hsi']
df_baru_hsi = df_baru_hsi.drop(columns=['Unnamed: 0'])
df_baru_hsi['path'] = path_hsi
df_baru_hsi['total_cost'] = total_cost_hsi
df_baru_hsi['checkpoint'] = checkpoint_hsi


df_baru_vpn = df_downlink[df_downlink['produk'] == 'vpn']
df_baru_vpn = df_baru_vpn.drop(columns=['Unnamed: 0'])
df_baru_vpn['path'] = path_vpn
df_baru_vpn['total_cost'] = total_cost_vpn

df_baru_transit = df_downlink[df_downlink['produk'] == 'transit']
df_baru_transit = df_baru_transit.drop(columns=['Unnamed: 0'])
df_baru_transit['path'] = path_transit
df_baru_transit['total_cost'] = total_cost_transit

df_baru_hsi

,from1,olt1,service,bw_down,produk,path,total_cost,checkpoint
0,CNGoogle,OLT00**KPS*3,MOBILE,0.068,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
1,CGW*.BDS,OLT00**KPS*3,MOBILE,0.136,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
2,CNFacebook,OLT00**KPS*3,MOBILE,0.068,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
3,CGW*.MDO,OLT00**KPS*3,MOBILE,0.136,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
4,CNNetflix,OLT00**KPS*3,MOBILE,0.068,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
5,CNConversant,OLT00**KPS*3,MOBILE,0.068,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
6,CNConversant,OLT00**KPS*3,WHOLESALE,0.034,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
7,CGW*.MDO,OLT00**KPS*3,WHOLESALE,0.102,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
8,CNGoogle,OLT00**KPS*3,WHOLESALE,0.034,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL
9,CNNetflix,OLT00**KPS*3,WHOLESALE,0.034,hsi,"[OLT00**KPS*3, MET**KPS, MET9**KBL]",30000,MET9**KBL


In [8]:
#DOWNLINK
path = []
costs = []
for i in range(len(df_baru_hsi)):
    sp = nx.shortest_path(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['from1'] , weight='weight')
    tc = nx.shortest_path_length(H, df_baru_hsi.iloc[i]['checkpoint'], df_baru_hsi.iloc[i]['from1'] , weight='weight')
    sp.pop(0)
    
    path.append(df_baru_hsi.iloc[i]['path'] + sp)
    costs.append(df_baru_hsi.iloc[i]['total_cost'] + tc)
    
df_akhir_hsi = df_baru_hsi.copy() 
df_akhir_hsi = df_akhir_hsi.drop('path', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('total_cost', axis=1)
df_akhir_hsi = df_akhir_hsi.drop('checkpoint', axis=1)
df_akhir_hsi['path'] = path
df_akhir_hsi['total_cost'] = costs

df_combined_downlink = pd.concat([df_akhir_hsi, df_baru_vpn, df_baru_transit])

path = []
for i in range(len(df_combined_downlink)):
    shortest_path = df_combined_downlink.iloc[i]['path']
    shortest_path = shortest_path[::-1]
    path.append(shortest_path)
    
df_combined_downlink['path'] = path

In [9]:
df_cleaning = df_cleaning.drop_duplicates()

df_cleaning = df_cleaning.rename({'from1': 'node', 'to1': 'neighbor'}, axis='columns')

# Membuat kolom kosong untuk menyimpan besaran total traffic yang melewati tiap edge
df_cleaning['inbound'] = 0
df_cleaning['outbound'] = 0
df_cleaning['dilewati'] = False
df_cleaning['status inbound'] = 'aman'
df_cleaning['status outbound'] = 'aman'
df_cleaning['total_loss_inbound'] = 0
df_cleaning['total_loss_outbound'] = 0

In [10]:
# Melakukan injeksi traffic uplink
custom_dict = {'MOBILE': 0, 'WHOLESALE': 1, 'EBIS': 2, 'RETAIL': 3}
df_combined_uplink = df_combined_uplink.sort_values(by=['service'], key=lambda x: x.map(custom_dict))
df_combined_uplink = df_combined_uplink.reset_index(drop=True)
df_combined_uplink['bw_final'] = df_combined_uplink['bw_up']
df_combined_uplink

# Iterasi untuk tiap traffic df_uplink
for i in range(len(df_combined_uplink)):

  # Melakukan konversi string shortest_path ke bentuk array
  shortest_path = str(df_combined_uplink.iloc[i]['path'])
  shortest_path = shortest_path[1:-1]
  # print(shortest_path)
  sp_array = list(shortest_path.split(", "))

  # Melakukan iterasi untuk tiap edge di shortest_path
  for j in range(len(sp_array)-1):

    # Mencari index edge yang sesuai di dataframe topologi (fin)
    idx_edge = (df_cleaning.loc[(df_cleaning['node'] == sp_array[j][1:-1])&(df_cleaning['neighbor']==sp_array[j+1][1:-1])].index.values)[0]
    idx_edge2 = (df_cleaning.loc[(df_cleaning['neighbor'] == sp_array[j][1:-1])&(df_cleaning['node']==sp_array[j+1][1:-1])].index.values)[0]
    # print("Edge antara "+ str(sp_array[j][1:-1])+" dan "+ str(sp_array[j+1][1:-1]))
    # print("Punya index: "+str(idx_edge)+"\n")

    # Perhitungan berapa besaran traffic yang bisa masuk ke dalam edge
    max_kap = float(df_cleaning.iloc[idx_edge]['kap'])
    cur_traffic = float(df_cleaning.iloc[idx_edge]['outbound'])
    bw_cur = float(df_combined_uplink.iloc[i]['bw_final'])
    df_cleaning.at[idx_edge, 'outbound'] += df_combined_uplink.iloc[i]['bw_final']
    df_cleaning.at[idx_edge,'dilewati'] = True
    if ((cur_traffic + bw_cur) > max_kap): # Jika jumlah
        df_cleaning.at[idx_edge, 'status outbound'] = 'loss'

    max_kap_reverse = float(df_cleaning.iloc[idx_edge2]['kap'])
    cur_traffic_reverse = float(df_cleaning.iloc[idx_edge2]['inbound'])
    bw_cur1 = float(df_combined_uplink.iloc[i]['bw_final'])
    df_cleaning.at[idx_edge2, 'inbound'] += df_combined_uplink.iloc[i]['bw_final']
    df_cleaning.at[idx_edge2,'dilewati'] = True
    if ((cur_traffic + bw_cur) > max_kap): # Jika jumlah
        df_cleaning.at[idx_edge2, 'status outbound'] = 'loss'

# for i in range(len(df_uplink)):
#   shortest_path = df_uplink.iloc[i]['shortest_path']
#   shortest_path = shortest_path[1:-1]
#   # print(shortest_path)
#   sp_array = list(shortest_path.split(", "))
#   for j in range(len(sp_array)-1):
#     # print(row[1]['olt1'])
#     # print(sp_array[i][1:-1])
#     # print(type(sp_array[i]))
#     idx_edge = (df.loc[(df['from1'] == sp_array[j][1:-1])&(df['to1']==sp_array[j+1][1:-1])].index.values)[0]
#     print("Edge antara "+ str(sp_array[j][1:-1])+" dan "+ str(sp_array[j+1][1:-1]))
#     print("Punya index: "+str(idx_edge))
#     df.at[idx_edge, 'current_kap'] += df_uplink.iloc[i]['bw_up']

#     print("BW terpakai: "+str(df_uplink.iloc[i]['bw_up'])+"\n")

df_cleaning = df_cleaning[df_cleaning.dilewati == True]
df_cleaning = df_cleaning.reset_index(drop=True)

In [11]:
df_cleaning

,node,neighbor,kap,jmllink,cost_link,inbound,outbound,dilewati,status inbound,status outbound,total_loss_inbound,total_loss_outbound
0,BR2**KBL,PE**KBL*HSI,520,7.0,0,0.000,0.744,True,aman,aman,0,0
1,OLT00**KPS*3,MET**KPS,6,6.0,0,0.000,0.744,True,aman,aman,0,0
2,BR2**KBL,MET9**KBL,850,40.0,0,0.744,0.000,True,aman,aman,0,0
3,MET9**KBL,MET**KPS,10,1.0,30000,0.744,0.000,True,aman,aman,0,0
4,MET**KPS,MET9**KBL,10,1.0,30000,0.000,0.744,True,aman,aman,0,0
5,PE**KBL*HSI,CNConversant,1377,153.0,0,0.000,0.056,True,aman,aman,0,0
6,PE**KBL*HSI,CNFacebook,6800,68.0,0,0.000,0.056,True,aman,aman,0,0
7,PE**KBL*HSI,CNGoogle,6120,306.0,0,0.000,0.056,True,aman,aman,0,0
8,PE**KBL*HSI,CNNetflix,6800,68.0,0,0.000,0.056,True,aman,aman,0,0
9,PE**RKT*HSI,CNIptv,20,2.0,0,0.000,0.030,True,aman,aman,0,0


In [12]:
df_combined_downlink['bw_final'] = df_combined_downlink['bw_down']
custom_dict = {'MOBILE': 0, 'WHOLESALE': 1, 'EBIS': 2, 'RETAIL': 3}
df_combined_downlink = df_combined_downlink.sort_values(by=['service'], key=lambda x: x.map(custom_dict))
df_combined_downlink = df_combined_downlink.reset_index(drop=True)
df_combined_downlink['bw_final'] = df_combined_downlink['bw_down']

for i in range(len(df_combined_downlink)):
  shortest_path = str(df_combined_downlink.iloc[i]['path'])
  shortest_path = shortest_path[1:-1]
  sp_array = list(shortest_path.split(", "))
  # print(sp_array)
  for j in range(len(sp_array)-1):
    # print(row[1]['olt1'])
    # print(sp_array[i][1:-1])
    # print(type(sp_array[i]))
    idx_edge = (df_cleaning.loc[(df_cleaning['node'] == sp_array[j][1:-1])&(df_cleaning['neighbor']==sp_array[j+1][1:-1])].index.values)[0]
    idx_edge2 = (df_cleaning.loc[(df_cleaning['neighbor'] == sp_array[j][1:-1])&(df_cleaning['node']==sp_array[j+1][1:-1])].index.values)[0]
    # print("Edge antara "+ str(sp_array[j][1:-1])+" dan "+ str(sp_array[j+1][1:-1]))
    # print("Punya index: "+str(idx_edge)+"\n")

    max_kap = float(df_cleaning.iloc[idx_edge]['kap'])
    cur_traffic = float(df_cleaning.iloc[idx_edge]['outbound'])
    bw_cur = float(df_combined_downlink.iloc[i]['bw_final'])
    df_cleaning.at[idx_edge, 'outbound'] += df_combined_downlink.iloc[i]['bw_final']
    df_cleaning.at[idx_edge,'dilewati'] = True
    if ((cur_traffic + bw_cur) > max_kap): # Jika jumlah
      df_cleaning.at[idx_edge, 'status outbound'] = 'loss'


    max_kap_reverse = float(df_cleaning.iloc[idx_edge2]['kap'])
    cur_traffic_reverse = float(df_cleaning.iloc[idx_edge2]['inbound'])
    bw_cur1 = float(df_combined_downlink.iloc[i]['bw_final'])
    df_cleaning.at[idx_edge2, 'inbound'] += df_combined_downlink.iloc[i]['bw_final']
    df_cleaning.at[idx_edge2,'dilewati'] = True
    if ((cur_traffic + bw_cur) > max_kap): # Jika jumlah
      df_cleaning.at[idx_edge2, 'status outbound'] = 'loss'
    
df_cleaning = df_cleaning[df_cleaning.dilewati == True]
df_cleaning = df_cleaning.reset_index(drop=True)

In [13]:
df_cleaning

,node,neighbor,kap,jmllink,cost_link,inbound,outbound,dilewati,status inbound,status outbound,total_loss_inbound,total_loss_outbound
0,BR2**KBL,PE**KBL*HSI,520,7.0,0,2.266,0.744,True,aman,aman,0,0
1,OLT00**KPS*3,MET**KPS,6,6.0,0,2.266,0.744,True,aman,aman,0,0
2,BR2**KBL,MET9**KBL,850,40.0,0,0.744,2.266,True,aman,aman,0,0
3,MET9**KBL,MET**KPS,10,1.0,30000,0.744,2.266,True,aman,aman,0,0
4,MET**KPS,MET9**KBL,10,1.0,30000,2.266,0.744,True,aman,aman,0,0
5,PE**KBL*HSI,CNConversant,1377,153.0,0,0.170,0.056,True,aman,aman,0,0
6,PE**KBL*HSI,CNFacebook,6800,68.0,0,0.170,0.056,True,aman,aman,0,0
7,PE**KBL*HSI,CNGoogle,6120,306.0,0,0.170,0.056,True,aman,aman,0,0
8,PE**KBL*HSI,CNNetflix,6800,68.0,0,0.170,0.056,True,aman,aman,0,0
9,PE**RKT*HSI,CNIptv,20,2.0,0,0.090,0.030,True,aman,aman,0,0


In [14]:
#Inject traffic uplink

import webbrowser
from pyvis import network as net

G = net.Network(notebook=True, directed=True, select_menu=True)

# Menambahkan simpul-simpul dari kolom 'from1'
for i in range(len(df_cleaning)):
    node = df_cleaning.iloc[i]['node']
    if "OLT" in node:
        warna='#FF0000'  # Merah untuk simpul dengan "OLT" di dalamnya
    elif "MET" in node:
        warna='#FFFF00'  # Biru untuk simpul lainnya
    elif "PE" in node or "P*" in node or "T" in node or "P*D1*PBR" in node:
        warna='#0000FF'
    if "BR2**KBL" in node or "BR2**RKT" in node:
        warna='#00FF00'
    if "CN" in node or "CGW" in node or "CGW2*.BTC" in node or "TEL#_CGW" in node:
        warna='#000000'

    G.add_node(node, color=warna)

# Menambahkan simpul-simpul dari kolom 'to1'


# Mengiterasi setiap baris pada dataframe
for i in range(len(df_cleaning)):
    from1_element = df_cleaning.iloc[i]['node']
    to1_element = df_cleaning.iloc[i]['neighbor']
    #cost_link = "Cost link: " + str(df_uplink.iloc[i]['cost_link']) + "\nKapasitas: " + str(df_uplink.iloc[i]['kap']) + "\nOutbound dari " + str(df_uplink.iloc[i]['node']) + " ke " + str(df_uplink.iloc[i]['neighbor']) +" : " + str(df_uplink.iloc[i]['inbound'])+ "\nInbound ke " + str(df_uplink.iloc[i]['neighbor']) + " dari " + str(df_uplink.iloc[i]['neighbor']) +  " : " + str(df_uplink.iloc[i]['inbound'])
    cost_link = "Cost link: " + str(df_cleaning.iloc[i]['cost_link']) + "\nKapasitas: " + str(df_cleaning.iloc[i]['kap']) + "\nKe arah " + str(df_cleaning.iloc[i]['neighbor']) + ": " + str(df_cleaning.iloc[i]['outbound']) + "\nTotal loss: " + str(df_cleaning.iloc[i]['total_loss_outbound'])
    
    if (df_cleaning.iloc[i]['outbound'] > df_cleaning.iloc[i]['kap'] ):
        G.add_edge(from1_element, to1_element, title=cost_link, color='#FF0000')  # Mengatur edge menjadi directed dengan arrows=True
    else:
        G.add_edge(from1_element, to1_element, title=cost_link, color='#000000')
        
    
# Menampilkan graf menggunakan Pyvis
G.show('graph_uplink.html')

webbrowser.open('graph_uplink.html',new=2)

graph_uplink.html


True